In [1]:
import pandas as pd
import altair as alt
import datetime as dt
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

# Inputs

In [2]:
sos_download_dir='/data2/elilouis/sublimationofsnow/sosnoqc'
DATE_FORMAT_STR = '%Y%m%d'

# Case study dates
start_date1 = '20230210'; end_date1 = '20230212'
start_date2 = '20230219'; end_date2 = '20230221'
start_date3 = '20230317'; end_date3 = '20230319' 

def format_date(date):
    return dt.datetime.strptime(
        date, 
        DATE_FORMAT_STR
    ).strftime(
        '%Y-%m-%d'
    )

start_date1f = format_date(start_date1); end_date1f = format_date(end_date1)
start_date2f = format_date(start_date2); end_date2f = format_date(end_date2)
start_date3f = format_date(start_date3); end_date3f = format_date(end_date3) 

In [3]:
df = pd.read_parquet(
    '~/sublimationofsnow/analysis/sos/tidy_df_30Min_20221201_20230501_noplanar_fit.parquet'
)
df['time'] = pd.to_datetime(df['time'])

In [4]:
ls -lah ~/sublimationofsnow/analysis/sos/tidy_df_30Min_20221201_20230501_noplanar_fit.parquet

-rw-rw-r--. 1 elilouis elilouis 26M May 23 21:41 /home/elilouis/sublimationofsnow/analysis/sos/tidy_df_30Min_20221201_20230501_noplanar_fit.parquet


In [11]:
import sys
sys.path.append("/home/elilouis/sublimationofsnow/")
import sosutils
from sostidy import SOSTidy

In [12]:
obukhov_src = df.query("measurement == 'Obukhov length'")

for tower in obukhov_src['tower'].unique():
    print(tower)
    for height in obukhov_src['height'].unique():
        print(height)
        src = obukhov_src.query(
                f"tower == '{tower}'"
            ).query(
                f"height == {height}"
            )
        df = SOSTidy.tidy_df_add_variable(
            df,
            src['height'] / src['value'],
            f"stability_function_{height}m_{tower}",
            "stability function",
            height,
            tower
        )

c
10.0
15.0
20.0
2.0
3.0
5.0


In [13]:
alt.Chart(df.query("measurement == 'richardson number'")).mark_boxplot(outliers=False).encode(
    alt.X("height:O"),
    alt.Y("value:Q").title('Ri')
).properties(width = 200, height = 200, title='Without outliers') | \
alt.Chart(df.query("measurement == 'richardson number'")).mark_boxplot(outliers={'size': 1}).encode(
    alt.X("height:O"),
    alt.Y("value:Q").title('Ri').scale(domain=[-100,100], clamp=True)
).properties(width = 200, height = 200, title='With outliers')

alt.HConcatChart(...)

In [14]:
alt.Chart(df.query("measurement == 'stability function'")).mark_boxplot(outliers=False).encode(
    alt.X("height:O"),
    alt.Y("value:Q").title('stability function')
).properties(width = 200, height = 200, title='Without outliers') | \
alt.Chart(df.query("measurement == 'stability function'")).mark_boxplot(outliers={'size': 1}).encode(
    alt.X("height:O"),
    alt.Y("value:Q").title('stability function').scale(domain=[-100,100], clamp=True)
).properties(width = 200, height = 200, title='With outliers')

alt.HConcatChart(...)

# Problem 1: Do Stable Atmospheric Conditions Shut Down Turbulence?

In [15]:
df1 = df.set_index('time').loc[start_date1:end_date1].reset_index()
df2 = df.set_index('time').loc[start_date2:end_date2].reset_index()
df3 = df.set_index('time').loc[start_date3:end_date3].reset_index()

Pick three different periods of 3-5 days in our dataset.

Assess:
* stability (temperature profiles), 
* the winds (wind speed and direction profiles and wind time series), 
* Richardson number, 
* the turbulent fluxes (TKE, sensible heat flux, latent heat fluxes) at two or more heights
                        
How are these related? What are the highest turbulent fluxes you observe during a period considered “strongly stable,” with a Richardson number greater than 0.2? (Hint: Histograms might be helpful here.)



In [16]:
def profiles_plot(df):
    profiles_df = df[df['time'].dt.hour % 4 == 0]
    profiles_df = profiles_df[profiles_df['time'].dt.minute == 0]
    profiles_df['day'] = profiles_df['time'].dt.day
    profiles_df['hour'] = profiles_df['time'].dt.hour

    src = profiles_df.query("measurement == 'potential virtual temperature'").query("tower == 'c'")
    temp_profile_plot = alt.Chart(src).mark_line().encode(
        alt.X("value:Q", sort='-y').title('Pot. Virtual Temp. (˚C)'),
        alt.Y("height:Q"),
        alt.Facet("day:O", columns=4),
        alt.Color("hour:O").scale(scheme='rainbow')
    ).properties(width=115, height=60).resolve_scale(x='independent')

    src = profiles_df.query("measurement == 'wind speed'").query("tower == 'c'")
    wind_profile_plot = alt.Chart(src).mark_line().encode(
        alt.X("value:Q", sort='-y').title('Wind speed (m/s)'),
        alt.Y("height:Q"),
        alt.Facet("day:O", columns=4, title=None).header(labelFontSize=0),
        alt.Color("hour:O").scale(scheme='rainbow')
    ).properties(width=115, height=60).resolve_scale(x='independent')

    src = profiles_df.query("measurement == 'wind direction'").query("tower == 'c'")
    winddir_profile_plot = alt.Chart(src).mark_line().encode(
        alt.X("value:Q", sort='-y').title('Wind direction (˚)'),
        alt.Y("height:Q"),
        alt.Facet("day:O", columns=4, title=None).header(labelFontSize=0),
        alt.Color("hour:O").scale(scheme='rainbow')
    ).properties(width=115, height=60).resolve_scale(x='independent')

    return (temp_profile_plot & wind_profile_plot & winddir_profile_plot)

In [17]:
profiles_plot(df1).properties(title=f'{start_date1f} - {end_date1f}') | profiles_plot(df2).properties(title=f'{start_date2f} - {end_date2f}') | profiles_plot(df3).properties(title=f'{start_date3f} - {end_date3f}')

alt.HConcatChart(...)

# Plot time series

In [18]:
def time_series_plot(src):
    wind_chart = alt.Chart(src).transform_filter(
        alt.datum.tower == 'c'
    ).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['wind speed'])
    ).mark_line().transform_window(
        rolling_mean='median(value)',
        frame=[-3, 3],
        groupby=['variable']
    ).encode(
            alt.X('time:T').title(None).axis(labels=False),
            alt.Y("rolling_mean:Q", title='Wind speed (m/s)').scale(domain=[0, 8]),
            alt.Color("height:O", scale=alt.Scale(scheme='rainbow'))
    ).properties(width=200, height=75)

    tke_chart = alt.Chart(src).transform_filter(
        alt.datum.tower == 'c'
    ).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['turbulent kinetic energy'])
    ).mark_line().transform_window(
        rolling_mean='median(value)',
        frame=[-3, 3],
        groupby=['variable']
    ).encode(
            alt.X('time:T').title(None).axis(labels=False),
            alt.Y("rolling_mean:Q", title='TKE (m²/s²)').scale(domain=[0,8]),
            alt.Color("height:O", scale=alt.Scale(scheme='rainbow'))
    ).properties(width=200, height=75)

    ri_chart = alt.Chart(src).transform_filter(
        alt.datum.tower == 'c'
    ).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['richardson number'])
    ).mark_line().encode(
            alt.X('time:T').title(None).axis(labels=False),
            alt.Y("value:Q", title='Ri').scale(domain=[-5, 5], clamp=True),
            alt.Color("height:O", scale=alt.Scale(scheme='rainbow'))
    ).properties(width=200, height=75)

    ri_bulk_chart = alt.Chart(src).transform_filter(
        alt.datum.tower == 'c'
    ).transform_filter(
        alt.FieldOneOfPredicate('variable', ['Ri_3m_c', 'RiB_3m_c'])
    ).mark_line().encode(
            alt.X('time:T').title(None).axis(labels=False),
            alt.Y("value:Q", title='Ri').scale(domain=[-5, 5], clamp=True),
            alt.StrokeDash("variable:O")
    ).properties(width=200, height=75)



    stability_chart = alt.Chart(src).transform_filter(
        alt.datum.tower == 'c'
    ).transform_filter(
        alt.FieldOneOfPredicate('measurement', ['stability function'])
    ).mark_line().encode(
            alt.X('time:T').title(None).axis(labels=False),
            alt.Y("value:Q", title='stability function').scale(domain=[-10, 10], clamp=True),
            alt.Color("height:O", scale=alt.Scale(scheme='rainbow'))
    ).properties(width=200, height=75)

    flux_chart = alt.Chart(src).transform_filter(
        alt.FieldOneOfPredicate('tower', ['c'])
    ).mark_line().transform_window(
        rolling_mean='median(value)',
        frame=[-3, 3],
        groupby=['variable']
    ).encode(
        alt.Color("height:O", scale=alt.Scale(scheme='rainbow'))
    ).properties(width=200, height=75)

    lh_flux_chart = flux_chart.transform_filter(
            alt.FieldOneOfPredicate('measurement', ['w_h2o_'])
        ).encode(
            alt.X('time:T').title(None).axis(labels=False),
            alt.Y("rolling_mean:Q", title='LH Flux (g/m²)').scale(domain=[-0.02, 0.02])
        ) 
    
    sh_flux_chart = flux_chart.transform_filter(
            alt.FieldOneOfPredicate('measurement', ['w_tc_'])
        ).encode(
            alt.X('time:T'),
            alt.Y("rolling_mean:Q", title='SH Flux (˚C * m/s)').scale(domain=[-0.06, 0.06])
        )


    return (wind_chart & tke_chart & ri_chart & ri_bulk_chart & lh_flux_chart & sh_flux_chart).resolve_scale(x='shared', strokeDash='independent')

In [19]:
time_series_plot(df1).properties(title=f'{start_date1f} - {end_date1f}') | time_series_plot(df2).properties(title=f'{start_date2f} - {end_date2f}') | time_series_plot(df3).properties(title=f'{start_date3f} - {end_date3f}')

alt.HConcatChart(...)

In [ ]:
rule = alt.Chart().mark_rule().encode(x=alt.datum(0.2))

def scatterplots(src):
    wide_df = src[src['variable'].isin(['Ri_3m_c', 'w_h2o__3m_c', 'w_tc__3m_c'])][
        ['variable', 'value', 'time']
    ].reset_index(drop=True).pivot(
        values=['value'], 
        columns='variable', 
        index='time'
    )

    wide_df.columns = ['Ri_3m_c', 'w_h2o__3m_c', 'w_tc__3m_c']
    wide_df = wide_df.reset_index()
    xscale = alt.Scale(domain = [-2, 4], clamp=True)
    yscale = alt.Scale(domain = [-.01, .01], clamp=True)

    base = alt.Chart(wide_df)
    points = base.mark_circle(size=10).encode(
        alt.X('Ri_3m_c').scale(xscale),
        alt.Y('w_h2o__3m_c').scale(yscale)
    ).properties(width=150, height=150)

    top_hist = base.mark_bar().encode(
        alt.X("Ri_3m_c")
        .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
        .title(None)
        .axis(labels=False),
        alt.Y('count()').title(None)
        
    ).properties(width=150, height=30)

    right_hist = base.mark_bar().encode(
        alt.Y("w_h2o__3m_c")
        .bin(maxbins=30, extent=yscale.domain)
        .title(None)
        .axis(labels=False),
        alt.X('count()').title(None)
    ).properties(width=30, height = 150)

    w_h2o_vs_ri_plot = top_hist & (points + rule | right_hist)
    

    xscale = alt.Scale(domain = [-2, 4], clamp=True)
    yscale = alt.Scale(domain = [-.05, .05], clamp=True)

    base = alt.Chart(wide_df)
    points = base.mark_circle(size=10).encode(
        alt.X('Ri_3m_c').scale(xscale),
        alt.Y('w_tc__3m_c').scale(yscale)
    ).properties(width=150, height=150)

    top_hist = base.mark_bar().encode(
        alt.X("Ri_3m_c")
        .bin(maxbins=30, extent=xscale.domain).scale(clamp=True)
        .title(None)
        .axis(labels=False),
        alt.Y('count()').title(None)
        
    ).properties(width=150, height=30)

    right_hist = base.mark_bar().encode(
        alt.Y("w_tc__3m_c")
        .bin(maxbins=30, extent=yscale.domain)
        .title(None)
        .axis(labels=False),
        alt.X('count()').title(None)
    ).properties(width=30, height = 150)

    

    w_tc_vs_ri_plot = (points + rule | right_hist)

    return (w_h2o_vs_ri_plot & w_tc_vs_ri_plot)

In [ ]:
scatterplots(df1).properties(title=f'{start_date1f} - {end_date1f}') | scatterplots(df2).properties(title=f'{start_date2f} - {end_date2f}') | scatterplots(df3).properties(title=f'{start_date3f} - {end_date3f}')

alt.HConcatChart(...)

# Problem 2: Mass vs Energy Fluxes in Sublimation

Consider again, the major wind event of 22 December 2021 that moved a fair bit of snow around. Over the 24-hour period of measured blowing snow at 1 m (as documented by the FlowCapt sensor), calculate 
* (a) the total mass (in grams/m^2) of snow that horizontally moved past the sensor, 
* (b) the total mass (in grams/m^2) of water vapor that was transported vertically away from the snow over this time interval, 
* (c) the total energy (in Joules, using the latent heat of sublimation) that went into converting snow to the amount of water vapor calculated in b, and 
* (d) the total energy change per m^2 area, in the snowpack, calculated with the specific heats of ice and air, the density of the snow to represent fractional ice content, the depth over which a change was observed, and the observed temperature change (as measured from the thermistors in the snowpack at this time)

In [ ]:
df4 = df.set_index('time').loc['20221222':'20221222'].reset_index()

## (A) Daily total blowing snow mass flux

In [ ]:
# values are in g/m^2/s over 30 minute windows
# to avoid filling in missing data points, take the mean
# multiply mean flux g/m^s/s times seconds in a day
mean_snow_flux = df4.query("variable == 'SF_avg_1m_ue'")['value'].mean()
daily_snow_flux = mean_snow_flux * 60*60*24

In [ ]:
mean_snow_flux, daily_snow_flux/1000

(8.138090542987314, 703.1310229141039)

## (B) Daily total vertical water vapor mass flux

In [ ]:
mean_LH_flux = df4.query("variable == 'w_h2o__2m_c'")['value'].mean()
daily_LH_flux = mean_LH_flux * 60*60*24

In [ ]:
mean_LH_flux

0.015498603305534037

In [ ]:
ls -lah | grep svg

-rw-rw-r--. 1 elilouis elilouis 409K May 23 02:49 case1.svg
-rw-rw-r--. 1 elilouis elilouis 413K May 23 02:49 case2.svg
-rw-rw-r--. 1 elilouis elilouis 414K May 23 02:50 case3.svg
-rw-rw-r--. 1 elilouis elilouis 467K May 23 22:23 visualization2.svg
-rw-rw-r--. 1 elilouis elilouis 377K May 23 22:24 visualization3.svg
-rw-rw-r--. 1 elilouis elilouis  36K May 23 14:08 visualization4.svg
-rw-rw-r--. 1 elilouis elilouis 227K May 23 22:23 visualization.svg


In [ ]:
daily_LH_flux/1000 #in units of kg

1.3390793255981408

## (C) Daily total vertical LH flux

In [ ]:
# latent heat of vaporization for the ice-vapor transition (sublimation)
L_s = 334 + 2256 # in units of "kJ/kg
L_s

2590

In [ ]:
(daily_LH_flux/1000) * L_s # in units of kJ

3468.215453299185

(D) Total energy change per m^2 area, in the snowpack,

Calculated with the specific heats of ice and air, the density of the snow to represent fractional ice content, the depth over which a change was observed, and the observed temperature change (as measured from the thermistors in the snowpack at this time)

In [ ]:
snow_temps_df = df4.query("measurement == 'snow temperature'").query("tower == 'uw'")
alt.Chart(snow_temps_df).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q").scale(zero=False).title("Temperature (˚C)"),
    alt.Color("height:O").scale(scheme='rainbow')
).properties(width=600, height=200)


alt.Chart(...)

Looking at the plot of the in-snow thermistor array, it looks like only the 0.4m measurement was buried the entire time, so that is the only temperature we will use to measure change. Note also that additional thermistors were buried during the day, but for simplicity, we will use the thermistors at the beginning of the day to estimate the snowpack depth of 45cm (it was between the 0.4 and 0.5 thermistors at the beginning of the day.).

We measure the change in temperature 0.4m to estimate the change of the snow's bulk temperature.

In [ ]:
# Assume 30% density
# Snowpack is 0.45 meters deep
# We saw an increase in temperature of 0.92˚C

specific_heat_ice = 2090 # Joules/kg/˚C 
specific_heat_air = 1005 # Joules/kg/˚C, dry air at sea level

air_density = df4.query("measurement == 'ait density'")['value'].mean() # kg/m^3
ice_density = 917 # kg/m^3

snowpack_temperature_change = df4.query("variable == 'Tsnow_0_4m_uw'")['value'].iloc[-1] - df4.query("variable == 'Tsnow_0_4m_uw'")['value'].iloc[0]

snow_density = 0.3
snow_depth = 0.45 # meters
area = 1 # meters^2
snow_volume = snow_depth*area # meters^3
air_volume = snow_volume*(1 - snow_density) # m^3
ice_volume = snow_volume*(snow_density) # m^3

air_mass = air_volume*air_density
ice_mass = ice_volume*ice_density

In [ ]:
air_mass, ice_mass

(nan, 123.795)

(D) Total energy change per m^2 area, in the snowpack,

Calculated with the specific heats of ice and air, the density of the snow to represent fractional ice content, the depth over which a change was observed, and the observed temperature change (as measured from the thermistors in the snowpack at this time)

snowpack energy change = 
        
        (change in bulk temperature of air)*(specific heat of air)*(mass of air) 
        + (change in bulk temperature of snow)*(specific heat of snow)*(mass of snow) 

Where we assume that (change in bulk temperature of air) = (change in bulk temperature of snow).

In [ ]:
snowpack_energy_change = snowpack_temperature_change*specific_heat_air*air_mass + snowpack_temperature_change*specific_heat_ice*ice_mass
snowpack_energy_change # in Joules

nan